# Simple Notion Connections

We want to:
- Connect via notion SDK
- Query db with a filter
- Create a test card and push it
- Edit hours on a card and push changes
- check that notion updates the last edited field


In [1]:
import requests
import json
import notion_client
import os
from dotenv import load_dotenv
load_dotenv()

NOTION_TOKEN = os.getenv('NOTION_TOKEN')
DATABASE_ID = os.getenv('DATABASE_ID')

headers = {
    "Authorization": "Bearer " + NOTION_TOKEN,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

#### Connect to notion and grab some pages

In [2]:
# manual request
num_pages = 10
get_url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
payload = {"page_size": num_pages}
response = requests.post(get_url, json=payload, headers=headers)

In [ ]:
response.json()['results'][0]['properties']['Tickets']['title'][0]['plain_text']

#### Query a db with a filter

will also use the notion python sdk this time

In [6]:
# with SDK
notion = notion_client.Client(auth=NOTION_TOKEN)

In [7]:
query_data = {
        "database_id": DATABASE_ID,
        "filter": {"property": "Tickets","title": {"contains": "Icebreaker"}}
    }

my_pages = notion.databases.query(**query_data).get("results")


In [ ]:
for page in my_pages:
    print(page['properties']['Tickets']['title'][0]['plain_text'])

In [ ]:
page['properties']['Sprint']['multi_select'][0]['name']

In [ ]:
page['properties']['Status']['status']['name']

Let's try filter on a sprint now instead

In [14]:
query_data = {
        "database_id": DATABASE_ID,
        "filter": {"property": "Sprint","multi_select": {"contains": "Sprint 51"}}
    }

my_pages = notion.databases.query(**query_data).get("results")


In [ ]:
len(my_pages)

In [ ]:
for page in my_pages:
    print(page['properties']['Tickets']['title'][0]['plain_text'])

Filter on a both a status and a sprint

In [38]:
query_data = {
    "database_id": DATABASE_ID,
    "filter": {
        "and": [
            {
                "property": "Sprint",
                "multi_select": {
                    "contains": "Sprint 51"
                }
            },
            {
                "property": "Status",
                "status": {
                    "equals": "In Progress"
                }
            }
        ]
    }
}

my_pages = notion.databases.query(**query_data).get("results")


Best way to structure it will probably be by building the JSON query/payload and then it's compatible with a direct request or using the notion client

In [ ]:
query_data

In [ ]:
for page in my_pages:
    print(page['properties']['Tickets']['title'][0]['plain_text'])

In [107]:
# Using a straight http post request

query_url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
payload = {"filter": {
        "and": [
            {
                "property": "Sprint",
                "multi_select": {
                    "contains": "Sprint 51"
                }
            },
            {
                "property": "Status",
                "status": {
                    "equals": "In Progress"
                }
            }
        ]
    }}
response = requests.post(get_url, json=payload, headers=headers)

my_pages = response.json()['results']

In [ ]:
for page in my_pages:
    print(page['properties']['Tickets']['title'][0]['plain_text'])

#### Create a test card and push it

In [78]:
test_page = { 
 'Time Assigned': {'id': 'LJ%40r', 'type': 'number', 'number': 69},
 'Tickets': {'id': 'title',
  'type': 'title',
  'title': [{'type': 'text',
    'text': {'content': 'Test Card', 'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': 'Test Card',
    'href': None}]}}

In [120]:
newcard = my_pages[0]["properties"].copy()

In [ ]:
newcard["Priority"]

In [ ]:
newcard["Priority"]

In [ ]:
newcard.keys()

In [ ]:
newcard

In [ ]:

newcard = my_pages[0]["properties"].copy()
# newcard = {"properties": newproperties}
newcard["Tickets"]["title"][0]["text"]["content"] = "Notime Icebreaker New"
newcard["Tickets"]["title"][0]["plain_text"] = "Notime Icebreaker New"

# Formulas need to be removed, will be auto-calculated and can't be set
keys_to_remove = ['Allocation', 'Status Number', 'Time Left', 'Priority Number', 'Hours Completed', 'Created']
for k in keys_to_remove:
    newcard.pop(k, None)

notion.pages.create(parent={"database_id": DATABASE_ID}, properties=newcard)


In [ ]:
newcard

In [ ]:
notion.pages.create(parent={"database_id": DATABASE_ID}, properties=newcard)

In [19]:
# Using a straight http post request

test_page = { 
 'Time Assigned': {'id': 'LJ%40r', 'type': 'number', 'number': 69},
 'Tickets': {'id': 'title',
  'type': 'title',
  'title': [{'type': 'text',
    'text': {'content': 'Test Card 2', 'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': 'Test Card 2',
    'href': None}]}}

create_url = f"https://api.notion.com/v1/pages"
payload = {
    "parent": {"database_id": DATABASE_ID},
    "properties": test_page
}
response = requests.post(create_url, json=payload, headers=headers)

In [ ]:
response

#### Edit hours on a card and push changes

In [ ]:
print(f"The card `{my_pages[0]['properties']['Tickets']['title'][0]['plain_text']}` currently has {my_pages[0]['properties']['Time Spent']['number']} hours logged")

In [22]:
# First with a http request PATCH call
page_id = my_pages[0]['id']

update_url = f"https://api.notion.com/v1/pages/{page_id}"
payload = {"properties": {"Time Spent": { "number": 2 }}}

response = requests.patch(update_url, headers=headers, json=payload)

In [ ]:
# let's fetch that card again and check the hours logged
updated_page = notion.pages.retrieve(page_id=id)

print(f"The card `{updated_page['properties']['Tickets']['title'][0]['plain_text']}` currently has {updated_page['properties']['Time Spent']['number']} hours logged")

In [ ]:
# and using the notion sdk
payload = {
    "page_id": id,
    "properties": {"Time Spent": { "number": 3 }}
}
notion.pages.update(**payload)

In [ ]:
# let's fetch that card again and check the hours logged
updated_page = notion.pages.retrieve(page_id=id)

print(f"The card `{updated_page['properties']['Tickets']['title'][0]['plain_text']}` currently has {updated_page['properties']['Time Spent']['number']} hours logged")

#### check that notion updates the last edited field

In [ ]:
updated_page['last_edited_time']

In [ ]:
updated_page['last_edited_by']